In [1]:
import os
import pandas as pd

In [2]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [3]:
folder("pcaps")

In [4]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [5]:
files_add=find_the_way('./','.pcap')
files_add

['./pcaps\\ACK.pcap']

In [6]:
for i in files_add:
    temp=i[8:]
    temp=temp.replace(" ","_")
    temp=temp.replace("\\","-")
    temp=f"./pcaps/{temp}"
    #os.rename(i,temp)
    print(temp)

./pcaps/ACK.pcap


In [7]:
files_add=find_the_way('./','.pcap')
uzun=len(files_add)
uzun

1

In [8]:
df = pd.read_csv( "dataset_description.csv")
df

,No,File Name,Category,Sub-category,Rule
0,2,ACK.pcap,DDoS,ACK FlooD,ip.dst==192.168.23.2 && tcp.flags.ack == 1


In [9]:
df.columns

Index(['No', 'File Name', 'Category', 'Sub-category', 'Rule'], dtype='object')

In [10]:
name=df['File Name']
rule=df['Rule']
cat=df['Category']
subcat=df['Sub-category']

In [11]:
name

0    ACK.pcap
Name: File Name, dtype: object

In [12]:
for i in range(len(name)):
    add=str(cat[i])+"@"+str(subcat[i])+"_"
    add=add.replace(" ","_")
    command='tshark -Y \"'+str(rule[i])+"\" -r ./pcaps/"+str(name[i])+" -T fields -e frame.number > ./pcaps/attack_"+str(name[i])[:-4]+"csv"
    os.system(command)
    command='tshark -Y \"!('+str(rule[i])+")\" -r ./pcaps/"+str(name[i])+" -T fields -e frame.number > ./pcaps/normal_"+str(name[i])[:-4]+"csv"
    os.system(command)
    files_add=find_the_way('./','.pcap')
    if uzun==len(files_add):    
        print(command,"\n")
    uzun=len(files_add) 
 
    

tshark -Y "!(ip.dst==192.168.23.2 && tcp.flags.ack == 1)" -r ./pcaps/ACK.pcap -T fields -e frame.number > ./pcaps/normal_ACK.csv 



# MERGE CSVS

In [13]:
name_list=find_the_way('./pcaps','.csv')

for i in name_list:
    boyut=os.path.getsize(i)
    if boyut==0:
        os.remove(i)
        
    

In [14]:
label_files=[]
for i in name_list:
    temp=i[15:]
    if temp not in label_files:
        label_files.append(temp)
    

In [15]:
for i in label_files:

    name=find_the_way('./pcaps',i)
    print(name)

    dfA = pd.read_csv(name[0], header=None)
    dfA=dfA.rename(columns={0: "PacketNumber"})
    dfA["Label"]=[1] * len(dfA)
    
    try:
        dfN = pd.read_csv(name[1], header=None)
        dfN=dfN.rename(columns={0: "PacketNumber"})    
        dfN["Label"]=[0] * len(dfN) 
        df = pd.concat([dfA,dfN])
        df=df.sort_values('PacketNumber')
        df.to_csv(f"./pcaps/{i}",  index=False)
        print(i,df.groupby("Label").size(),"\n\n\n")
        os.remove(f"./pcaps\\normal_{i}")
        os.remove(f"./pcaps\\attack_{i}")
    except:
        dfA.to_csv(f"./pcaps/{i}",  index=False)
        print(i,dfA.groupby("Label").size(),"\n\n\n")
        

        


['./pcaps\\attack_ACK.csv', './pcaps\\normal_ACK.csv']
ACK.csv Label
0    1712900
1    1724690
dtype: int64 





In [16]:
#!shutdown /s /t 36

In [17]:
df

,PacketNumber,Label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1712898,3437586,0
1724687,3437587,1
1724688,3437588,1
1712899,3437589,0
